In [23]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')

In [24]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

d:\Projects\E2E\e2e\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\E2E\e2e\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Projects\E2E\e2e\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
loader1=WebBaseLoader("https://www.analyticsvidhya.com/blog/2023/05/build-a-chatgpt-for-pdfs-with-langchain/")
docs=loader1.load()
documnets=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorstore1=FAISS.from_documents(documnets,huggingface_embeddings)
retriver1=vectorstore1.as_retriever()
retriver1

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000202D22331C0>)

In [26]:
loader2=PyPDFLoader("acsbr-016.pdf")
docs2=loader2.load()
documents2=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs2)
vectorstore2=FAISS.from_documents(documents2,huggingface_embeddings)
retriver12=vectorstore2.as_retriever()
retriver12


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002031BC93D90>)

In [8]:
retriever_tool_pdf=create_retriever_tool(retriver1,"pdf_search",
                      "Search for information about the question asked in the pdf search tool, if you find anything you can use that information or else you can search in another tools")

In [9]:
retriever_tool_url=create_retriever_tool(retriver12,"website_search",
                      "Search for information about the question asked in the website search tool, if you find anything you can use that information or else you can search in another tools")

In [27]:
retriever_tool_pdf.name, retriever_tool_url.name

('pdf_search', 'website_search')

In [28]:
retriever_tool_url

Tool(name='website_search', description='Search for information about the question asked in the website search tool, if you find anything you can use that information or else you can search in another tools', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000202D2128700>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000202D2C01C60>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000202D21288B0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000202D2C01C60>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [29]:
tools=[retriever_tool_pdf,retriever_tool_url]

In [30]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name='llama3-8b-8192')


In [31]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
agent = initialize_agent(llm=llm,
                         tools=tools,
                         prompt=prompt,
                         agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=True,
                         output_key = "result",
                         handle_parsing_errors = True,
                         max_iterations=3,
                         early_stopping_method="generate",
                         memory = ConversationBufferMemory(memory_key = 'chat_history')                    
                        )

In [33]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(memory=ConversationBufferMemory(memory_key='chat_history'), verbose=True, tags=['conversational-react-description'], agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate

In [35]:
response=agent.invoke({"input":"Tell me about langchain"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Langchain is a type of large language model trained by Meta AI, designed to generate human-like text and engage in natural-sounding conversations. It is a type of transformer-based language model that uses a multi-layer perceptron (MLP) to generate text. Langchain is trained on a large corpus of text data, including books, articles, and websites, and is able to learn patterns and relationships in the data to generate coherent and relevant responses. It is also capable of generating text in different styles and formats, such as news articles, stories, and even entire books.

> Finished chain.


In [37]:
response['output']

'Langchain is a type of large language model trained by Meta AI, designed to generate human-like text and engage in natural-sounding conversations. It is a type of transformer-based language model that uses a multi-layer perceptron (MLP) to generate text. Langchain is trained on a large corpus of text data, including books, articles, and websites, and is able to learn patterns and relationships in the data to generate coherent and relevant responses. It is also capable of generating text in different styles and formats, such as news articles, stories, and even entire books.'

In [38]:
llm_model="llama3-8b-8192"

In [39]:
llm=ChatGroq(groq_api_key="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",
             
             model_name=llm_model)

In [40]:
initialize_agent(llm=llm,
                         tools=tools,
                         prompt=prompt,
                         agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=True,
                         output_key = "result",
                         handle_parsing_errors = True,
                         max_iterations=3,
                         early_stopping_method="generate",
                         memory = ConversationBufferMemory(memory_key = 'chat_history')                    
                        )

AgentExecutor(memory=ConversationBufferMemory(memory_key='chat_history'), verbose=True, tags=['conversational-react-description'], agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. 